In [2]:
import numpy as np
import matplotlib.pyplot as plt

data = np.load('/home/exx/projects/manifolds/OMG-Planner/output_videos/dbg/scene_3/data.npy', allow_pickle=True, encoding='latin1')

In [4]:
rew, info, plan = data

1